In [ ]:
# USAGE
# python3 finetune_boats.py --dataset ../datasets/boats/images \
# 	--model boats.model

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from pyimagesearch.preprocessing import ImageToArrayPreprocessor
from pyimagesearch.preprocessing import AspectAwarePreprocessor
from pyimagesearch.datasets import SimpleDatasetLoader
from pyimagesearch.nn.conv import FCHeadNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import pandas as pd
from imutils import paths
import numpy as np
import argparse
import os
from datetime import datetime
start_time = datetime.now()
# do your work here

dataset = '../datasets/boats/images/'
model_output_path = 'git_output/boats/boats_fine_tune.model'
figure_output_path = 'git_output/boats/boats_fine_tune_plot.png'
csv_init_output_path = 'git_output/boats/boats_fine_tune_head.csv'
csv_final_output_path = 'git_output/boats/boats_fine_tune_final.csv'
NUM_INIT_EPOCHS = 25
NUM_TUNE_EPOCHS = 100
DIMS = (224, 224)

# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

# grab the list of images that we'll be describing, then extract
# the class label names from the image paths
print("[INFO] loading images...")
imagePaths = list(paths.list_images(dataset))
classNames = [pt.split(os.path.sep)[-2] for pt in imagePaths]
classNames = [str(x) for x in np.unique(classNames)]

# initialize the image preprocessors
aap = AspectAwarePreprocessor(DIMS[0], DIMS[1])
iap = ImageToArrayPreprocessor()

# load the dataset from disk then scale the raw pixel intensities to
# the range [0, 1]
sdl = SimpleDatasetLoader(preprocessors=[aap, iap])
(data, labels) = sdl.load(imagePaths, verbose=500)
data = data.astype("float") / 255.0

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.25, random_state=42)

# convert the labels from integers to vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

# load the VGG16 network, ensuring the head FC layer sets are left
# off
baseModel = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(DIMS[0], DIMS[1], 3)))

# initialize the new head of the network, a set of FC layers
# followed by a softmax classifier
headModel = FCHeadNet.build(baseModel, len(classNames), 256)

# place the head FC model on top of the base model -- this will
# become the actual model we will train
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they
# will *not* be updated during the training process
for layer in baseModel.layers:
	layer.trainable = False

# compile our model (this needs to be done after our setting our
# layers to being non-trainable
print("[INFO] compiling model...")
opt = RMSprop(lr=0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the head of the network for a few epochs (all other
# layers are frozen) -- this will allow the new FC layers to
# start to become initialized with actual "learned" values
# versus pure random
print("[INFO] training head...")
model.fit_generator(aug.flow(trainX, trainY, batch_size=32),
	validation_data=(testX, testY), epochs=NUM_INIT_EPOCHS,
	steps_per_epoch=len(trainX) // 32, verbose=1)

# evaluate the network after initialization
print("[INFO] evaluating after initialization...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=classNames))

# output classification report to CSV
report = classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), 
    output_dict=True, target_names=classNames)
df = pd.DataFrame(report).transpose()
df.to_csv(csv_init_output_path)

# now that the head FC layers have been trained/initialized, lets
# unfreeze the final set of CONV layers and make them trainable
for layer in baseModel.layers[15:]:
	layer.trainable = True

# for the changes to the model to take affect we need to recompile
# the model, this time using SGD with a *very* small learning rate
print("[INFO] re-compiling model...")
opt = SGD(lr=0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the model again, this time fine-tuning *both* the final set
# of CONV layers along with our set of FC layers
print("[INFO] fine-tuning model...")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=32),
	validation_data=(testX, testY), epochs=NUM_TUNE_EPOCHS,
	steps_per_epoch=len(trainX) // 32, verbose=1)

# evaluate the network on the fine-tuned model
print("[INFO] evaluating after fine-tuning...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=classNames))

# output classification report to CSV
report = classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), 
    output_dict=True, target_names=classNames)
df = pd.DataFrame(report).transpose()
df.to_csv(csv_final_output_path)

end_time = datetime.now()
print('*'* 50, '\n[INFO] Duration: {}'.format(end_time - start_time), '\n', '*'*50)

# save the network to disk
print("[INFO] serializing network to...{}".format(model_output_path))
model.save(model_output_path)

# plot the training loss and accuracy
N = np.arange(0, NUM_TUNE_EPOCHS)
plt.style.use("ggplot")
plt.figure(figsize=(12, 8))
plt.ylim(None, 2)
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper right")
plt.savefig(figure_output_path)
plt.show()

[INFO] loading images...
[INFO] processed 500/693
[INFO] compiling model...
[INFO] training head...
Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 16 steps, validate on 174 samples
Epoch 1/25
16/16 [==============================] - 105s 7s/step - loss: 5.6297 - accuracy: 0.3696 - val_loss: 1.0544 - val_accuracy: 0.5287
Epoch 2/25
16/16 [==============================] - 97s 6s/step - loss: 1.0770 - accuracy: 0.5257 - val_loss: 0.6732 - val_accuracy: 0.7701
Epoch 3/25
16/16 [==============================] - 92s 6s/step - loss: 1.1407 - accuracy: 0.5421 - val_loss: 0.9235 - val_accuracy: 0.5460
Epoch 4/25
16/16 [==============================] - 90s 6s/step - loss: 1.0136 - accuracy: 0.5421 - val_loss: 0.6491 - val_accuracy: 0.7356
Epoch 5/25
16/16 [==============================] - 90s 6s/step - loss: 0.9177 - accuracy: 0.5791 - val_loss: 0.5588 - val_accuracy: 0.8161
Epoch 6/25
16/16 [==============================] - 90